Problem 1: Production economy and CO2 taxation

In [1]:
from types import SimpleNamespace
import numpy as np
from scipy.optimize import minimize, root

In [2]:
par = SimpleNamespace()
# firms
par.A = 1.0
par.gamma = 0.5
# households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
# government
par.tau = 0.0
par.T = 0.0
# Question 3
par.kappa = 0.1

#Set w = 1 as numearie
par.w = 1

Define model

In [3]:

def l1_star(par, p1):
    return (p1 * par.A * par.gamma/par.w)**(1 / (1 - par.gamma)) 

def l2_star(par, p2):
    return (p2 * par.A * par.gamma/par.w)**(1 / (1 - par.gamma))

def y1_star(par, l1):
    return par.A * (l1_star(par,p1))**par.gamma

def y2_star(par, l2):
    return par.A * (l2_star(par,p2))**par.gamma

def pi1_star(par, p1):
    return ((1 - par.gamma) / par.gamma) * par.w * (p1 * par.A * par.gamma / par.w)**(1 / (1 - par.gamma))

def pi2_star(par, p2):
    return ((1 - par.gamma) / par.gamma) * par.w * (p2 * par.A * par.gamma / par.w)**(1 / (1 - par.gamma))

def c1(l, par, p1, p2):
    return par.alpha * ((par.w * l + par.T + pi1_star(par, p1) + pi2_star(par, p2)) / p1)

def c2(l, par, p1, p2):
    return (1 - par.alpha) * ((par.w * l + par.T + pi1_star(par, p1) + pi2_star(par, p2)) / (p2 + par.tau))

def l_star(par, p1, p2, initial_guess=0.1):
    def negative_objective(l):
        c1_val = c1(l, par, p1, p2)
        c2_val = c2(l, par, p1, p2)
        if c1_val <= 0 or c2_val <= 0:
            return np.inf  # Penalize invalid solutions
        return -((np.log(max(c1_val, 1e-8))**par.alpha * max(c2_val, 1e-8)**(1 - par.alpha)) - par.nu * l**(1 + par.epsilon) / (1 + par.epsilon))

    result = minimize(negative_objective, initial_guess, method='BFGS')

    # Extract the optimal value of l
    l_star_value = result.x[0]

    return l_star_value


def c1_star(l_star_value, par, p1, p2):
    return c1(l_star_value, par, p1, p2)

def c2_star(l_star_value, par, p1, p2):
    return c2(l_star_value, par, p1, p2)



Define excess demand

In [4]:
def excessdemand_labor(par, p1, p2):
    l1 = l1_star(par, p1)
    l2 = l2_star(par, p2)
    l_opt = l_star(par, p1, p2)
    return l1 + l2 - l_opt


def excessdemand_goodmarket1(par, p1, p2):
    l1 = l1_star(par, p1)
    y1 = y1_star(par, l1)
    l_opt = l_star(par, p1, p2)
    c1_opt = c1_star(l_opt, par, p1, p2)
    return y1 - c1_opt

def excessdemand_goodmarket2(par, p1, p2):
    l2 = l2_star(par, p2)
    y2 = y2_star(par, l2)
    l_opt = l_star(par, p1, p2)
    c2_opt = c2_star(l_opt, par, p1, p2)
    return y2 - c2_opt

Find the excess demand for different p1 and p2 values

In [5]:
# Example p1 and p2 values
p1_values = np.linspace(0.5, 1.5, 10)
p2_values = np.linspace(0.5, 1.5, 10)

# Initialize arrays to store results
excess_labor_results = np.zeros((len(p1_values), len(p2_values)))
excess_goodmarket1_results = np.zeros((len(p1_values), len(p2_values)))
excess_goodmarket2_results = np.zeros((len(p1_values), len(p2_values)))



# Calculate excess demand for each combination of p1 and p2
for i, p1 in enumerate(p1_values):
    for j, p2 in enumerate(p2_values):
        excess_labor_results[i, j] = excessdemand_labor(par, p1, p2)
        excess_goodmarket1_results[i, j] = excessdemand_goodmarket1(par, p1, p2)
        excess_goodmarket2_results[i, j] = excessdemand_goodmarket2(par, p1, p2)

C:\Users\asv\AppData\Local\Temp\ipykernel_24820\3727888817.py:31: RuntimeWarning: invalid value encountered in power
  return -((np.log(max(c1_val, 1e-8))**par.alpha * max(c2_val, 1e-8)**(1 - par.alpha)) - par.nu * l**(1 + par.epsilon) / (1 + par.epsilon))


In [6]:
print("Excess labor demand:\n", excess_labor_results)
print("Excess good market 1 demand:\n", excess_goodmarket1_results)
print("Excess good market 2 demand:\n", excess_goodmarket2_results)

Excess labor demand:
 [[0.025      0.0558642  0.09290123 0.13611111 0.18549383 0.24104938
  0.30277778 0.37067901 0.44475309 0.525     ]
 [0.0558642  0.0867284  0.12376543 0.16697531 0.21635802 0.27191358
  0.33364198 0.40154321 0.47561728 0.5558642 ]
 [0.09290123 0.12376543 0.16080247 0.20401235 0.25339506 0.30895062
  0.37067901 0.43858025 0.51265432 0.59290123]
 [0.13611111 0.16697531 0.20401235 0.24722222 0.29660494 0.35216049
  0.41388889 0.48179012 0.5558642  0.63611111]
 [0.18549383 0.21635802 0.25339506 0.29660494 0.34598765 0.40154321
  0.4632716  0.53117284 0.60524691 0.68549383]
 [0.24104938 0.27191358 0.30895062 0.35216049 0.40154321 0.45709877
  0.51882716 0.5867284  0.66080247 0.74104938]
 [0.30277778 0.33364198 0.37067901 0.41388889 0.4632716  0.51882716
  0.58055556 0.64845679 0.72253086 0.80277778]
 [0.37067901 0.40154321 0.43858025 0.48179012 0.53117284 0.5867284
  0.64845679 0.71635802 0.7904321  0.87067901]
 [0.44475309 0.47561728 0.51265432 0.5558642  0.60524691 0.

Question 2: Find p1 and p2

In [7]:
def objective(p):
    p1, p2 = p
    return [excessdemand_labor(par, p1, p2), 0]  # Adding a second element to match the expected output shape

# Initial guess for p1 and p2
initial_guess = [1.0, 1.5]

# Find the values of p1 and p2 that make excessdemand_labor equal to zero
result = root(objective, initial_guess)

if result.success:
    p1_opt, p2_opt = result.x
    print(f"The values of p1 and p2 that give excessdemand_labor = 0 are: p1 = {p1_opt}, p2 = {p2_opt}")
else:
    print("The root-finding algorithm did not converge")

The values of p1 and p2 that give excessdemand_labor = 0 are: p1 = -0.020378704051218454, p2 = 0.6321271299518744


C:\Users\asv\AppData\Local\Temp\ipykernel_24820\3727888817.py:31: RuntimeWarning: invalid value encountered in power
  return -((np.log(max(c1_val, 1e-8))**par.alpha * max(c2_val, 1e-8)**(1 - par.alpha)) - par.nu * l**(1 + par.epsilon) / (1 + par.epsilon))
c:\Users\asv\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Question 3: What values of tau and T should the government choose to maximize SWF?

In [8]:

def U():

    def negative_utility(c1, c2, l, p1, p2, par):
        consumption = log(c1(par, l, p1, p2)**par.alpha*c2(par, l, p1,p2)**(1-par.alpha))
        work = par.nu*l**(1-par.epsilon)/(1+par.epsilon)
        u = -(consumption - work)

    # Define the budget constraint
    constraint = p1 * c1 + (p2 + tau) * c2 - (w * ell + T + pi1_star + pi2_star)

    # Initial guess
    x0 = 1S

    # Minimize the objective function with the constraint
    result = minimize(objective, x0, constraints={'type': 'eq', 'fun': constraint, 'args': (params['p1'], params['p2'], params['w'], params['tau'], params['T'], params['alpha'], params['nu'], params['epsilon'], params['pi1_star'], params['pi2_star'])})

    # Print the results
    print("Optimization Result:", result)
    print(f"c1: {result.x[0]}, c2: {result.x[1]}, ell: {result.x[2]}")




def SWF(U, par, y2_star):

    return U - par.kappa * y2_star
